In [4]:
def funcWACC (sticker):
    # WACC Calculator for a given stock
    
    # Cost of Debt Calculation not tax exempted
    
    # Read longTermDebt
    dframe_Balance_Sheet = si.get_balance_sheet(sticker)
    longTermDebt_col = dframe_Balance_Sheet.loc['longTermDebt',:]
    longTermDebt = longTermDebt_col[0] / 1000000 #convert to millions
    shortTermDebt = 0
    
    # Read interestExpense, incomeTaxExpense, incomeBeforeTax
    dframe_Income_Statement = si.get_income_statement(sticker)
    interestExpense_col = dframe_Income_Statement.loc['interestExpense',:]
    interestExpense = abs(interestExpense_col[0]) / 1000000 #convert to millions
    incomeTaxExpense_col = dframe_Income_Statement.loc['incomeTaxExpense', :]
    incomeTaxExpense = abs(incomeTaxExpense_col[0]) / 1000000 #convert to millions
    incomeBeforeTax_col = dframe_Income_Statement.loc['incomeBeforeTax', :]
    incomeBeforeTax = abs(incomeBeforeTax_col[0]) / 1000000 #convert to millions

    CostOfDebt_PreTax = interestExpense/longTermDebt
    
    # Effective Tax Rate Calculation
    EffTaxRate = incomeTaxExpense/incomeBeforeTax
    
    # Cost of Debt * (1-T) Tax Exmpted
    CostOfDebt = CostOfDebt_PreTax * (1-EffTaxRate)
    
    # Cost of Equity Calculation
    
    # Read Risk Free Interest Rate - Treasury Yield 10 Years
    url_bond = 'https://finance.yahoo.com/bonds'
    dframe_list_Bond = pd.read_html(url_bond)
    TreasRate = dframe_list_Bond[0]
    RiskFreeRate = TreasRate.loc[2,'Last Price']/100
    
    # Read Beta
    dframe_Stats = si.get_stats(sticker)
    BetaVal = float(dframe_Stats.loc[0,'Value']) #data is in str, convert to float
    
    #MarketReturn = 8.5%
    MarketReturn = 0.085
    
    CostOfEquity = RiskFreeRate + (BetaVal * (MarketReturn - RiskFreeRate))
    
    # Weight of Debt and Equity Calculation
    
    TotalDebt = longTermDebt + shortTermDebt
    
    # Read Market Cap
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    MarketCap_tab = dframe_list_Summary[1]
    MarketCap_str = MarketCap_tab.loc[0,1]
    if MarketCap_str[-1:]=='B':    #check if Billion
        MarketCap = float(MarketCap_str[:-1])*1000
    if MarketCap_str[-1:]=='T':    #check if Trillion
        MarketCap = float(MarketCap_str[:-1])*1000*1000
    
    # Weight of Debt
    WeightOfDebt = TotalDebt / (TotalDebt + MarketCap)
    WeightOfEquity = MarketCap / (TotalDebt + MarketCap)
    
    # Weighted Average Cost of Capital (WACC) Calculation
    
    WACC = (CostOfDebt * WeightOfDebt) + (CostOfEquity * WeightOfEquity)
    
    return WACC